<h1> Imports

Make sure to download fmnist in data/fashion

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
fmnist = input_data.read_data_sets("/data/fashion", one_hot=True)

<h1> Parameters

In [0]:
lr_param = 0.001
epochs = 50
batch_size = 32

image_dimension = 784
neural_network_dimension = 512

latent_variable_dimension = 10

<h3>Initialize</h3>

In [0]:
def xavier(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1/tf.sqrt(in_shape[0]/2.0))
  return(val)

<h3> Weights and Biases</h3>

In [0]:
Weight = { "weight_matrix_encoder_hidden" : tf.Variable(xavier([image_dimension, neural_network_dimension])),
          "weight_mean_hidden" : tf.Variable(xavier([neural_network_dimension, latent_variable_dimension])),
          "weight_std_hidden" : tf.Variable(xavier([neural_network_dimension, latent_variable_dimension])),
          "weight_matrix_decoder_hidden" : tf.Variable(xavier([latent_variable_dimension, neural_network_dimension])),
          "weight_decoder" : tf.Variable(xavier([neural_network_dimension, image_dimension])),
         }
Bias = {"bias_matrix_encoder_hidden" : tf.Variable(xavier([neural_network_dimension])),
          "bias_mean_hidden" : tf.Variable(xavier([latent_variable_dimension])),
          "bias_std_hidden" : tf.Variable(xavier([latent_variable_dimension])),
          "bias_matrix_decoder_hidden" : tf.Variable(xavier([neural_network_dimension])),
          "bias_decoder" : tf.Variable(xavier([image_dimension])),
         }

<h1> Encoder

In [0]:
image_x = tf.placeholder(tf.float32, shape = [None, image_dimension])

Encoder_layer = tf.add(tf.matmul(image_x, Weight["weight_matrix_encoder_hidden"]),Bias["bias_matrix_encoder_hidden"])
Encoder_layer = tf.nn.tanh(Encoder_layer)

Mean_layer = tf.add(tf.matmul(Encoder_layer, Weight["weight_mean_hidden"]), Bias["bias_mean_hidden"])
Standard_deviation_layer = tf.add(tf.matmul(Encoder_layer, Weight["weight_std_hidden"]), Bias["bias_std_hidden"])

<h1> Reparameterization Trick

In [0]:
epsilon = tf.random_normal(tf.shape(Standard_deviation_layer), dtype = tf.float32, mean= 0.0, stddev = 1.0)
latent_layer = Mean_layer*tf.exp(0.5*Standard_deviation_layer)*epsilon

<h1> Decoder

In [0]:
Decoder_hidden = tf.add(tf.matmul(latent_layer, Weight["weight_matrix_decoder_hidden"]),Bias["bias_matrix_decoder_hidden"])
Decoder_hidden = tf.nn.tanh(Decoder_hidden)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden, Weight["weight_decoder"]), Bias["bias_decoder"])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

<h2> Loss

In [0]:
def loss_function(original_image, reconstructed_image):
  
  # Reconstruction Loss
  data_fidelity_loss = original_image * tf.log(1e-10 + reconstructed_image) + (1-original_image)*tf.log(1e-10 + 1- reconstructed_image)
  data_fidelity_loss = -tf.reduce_sum(data_fidelity_loss, 1)
  
  # KL Divergence loss
  KL_div_loss = 1+Standard_deviation_layer - tf.square(Mean_layer) - tf.exp(Standard_deviation_layer)
  KL_div_loss = -0.5 * tf.reduce_sum(KL_div_loss, 1)
  
  alpha = 1
  beta = 1
  network_loss = tf.reduce_mean(alpha*data_fidelity_loss + beta*KL_div_loss)
  
  return(network_loss)

loss_value = loss_function(image_x, Decoder_output_layer)

optimizer =tf.train.RMSPropOptimizer(lr_param).minimize(loss_value)

<h2> Train

In [0]:
init = tf.global_variables_initializer()

sess = tf.Session()

sess.run(init)

for i in range(epochs):
    x_batch,_ = fmnist.train.next_batch(batch_size)
    _, loss = sess.run([optimizer, loss_value], feed_dict = {image_x: x_batch})
  
    
  print("Loss is {0} at iteration {1}".format(loss, i))

<h2> Test

In [0]:
noise_x = tf.placeholder(tf.float32, shape = [None, latent_variable_dimension])

Decoder_hidden = tf.add(tf.matmul(latent_layer, Weight["weight_matrix_decoder_hidden"]),Bias["bias_matrix_decoder_hidden"])
Decoder_hidden = tf.nn.tanh(Decoder_hidden)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden, Weight["weight_decoder"]), Bias["bias_decoder"])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

<h3> Output Visualization</h3>

In [0]:
n = 20
x_axis = np.linspace(-2, 2, n)
y_axis = np.linspace(-2, 2, n)

canvas = np.empty((28 * n, 28 * n))
for i, yi in enumerate(x_axis):
    for j, xi in enumerate(y_axis):
        z_mu = np.array([[xi, yi]] * batch_size)
        x_mean = sess.run(Decoder_output_layer, feed_dict={noise_input: z_mu})
        canvas[(n - i - 1) * 28:(n - i) * 28, j * 28:(j + 1) * 28] = \
        x_mean[0].reshape(28, 28)

plt.figure(figsize=(8, 10))
Xi, Yi = np.meshgrid(x_axis, y_axis)
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()